In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !pip install gdown

# import gdown
# url_data = 'https://drive.google.com/file/d/1uqvV0_u3LbU-sDn0-epFH6NiWr4_uK4c/view?usp=share_link'
# gdown.download(url=url_data, output="data.zip", quiet=False, fuzzy=True)

# import zipfile
# with zipfile.ZipFile('data.zip', 'r') as zip_ref:
#     zip_ref.extractall('data')

In [3]:
import os
os.listdir('data/')

['README.roboflow.txt',
 'valid',
 'README.dataset.txt',
 'test',
 'data.yaml',
 'train']

In [4]:
from yolov5.train import train
from yolov5.detect import run

In [5]:
# !python yolov5/train.py --img 256 --batch 16 --epochs 5 --data data/data.yaml --weights yolov5s.pt

## Model Wrapper

In [6]:
import shutil
import typing as tp
import tempfile


def removesuffix(line, suffixes):
    for suffix in suffixes:
        if line.endswith(suffix):
            return line[:-len(suffix)]
        

class YoloWrapper():
    def __init__(self, weights,
                 data_yaml='data/data.yaml',
                 img_size=(256, 256),
                 save_txt=True,
                 project='preds',
                 exist_ok=False,
                 name='') -> None:
        self.weights = weights
        self.data_yaml = data_yaml
        self.img_size = img_size
        self.save_txt = save_txt
        self.project = project
        self.exist_ok = exist_ok
        self.name = name
        
    @staticmethod
    def dct_preds(line):
        splited_line = line.strip().split()
        res = {'class_id':splited_line[0],
                'center_x': float(splited_line[1]),
                'center_y': float(splited_line[2]),
                'width': float(splited_line[3]),
                'height': float(splited_line[4])}
        if len(splited_line) == 6:
            res.update({'conf': float(splited_line[5])})
        return res
        
    def _predict(self, source, data, imgsz, save_path):
        shutil.rmtree(save_path)
        run(weights=self.weights, source=source, data=data,
            imgsz=imgsz, save_txt=True, project='preds',
            exist_ok=False, name='', save_conf=True)
        
    def predict(self, X: tp.List[str]):
        with tempfile.NamedTemporaryFile(mode='w', suffix='.txt') as fp:
            fp.write("\n".join(X))
            fp.seek(0)
            self._predict(fp.name, self.data_yaml, self.img_size, self.project)

        preds = {}
        for label in os.listdir(os.path.join(self.project, 'labels')):
            path_to_label = os.path.join(self.project, 'labels', label)
            
            with open(path_to_label, 'r') as f:
                _, img_name = path_to_label.rsplit('/', maxsplit=1)
                img_name = removesuffix(img_name, ['.txt']) + '.jpg'
                preds[img_name] = [self.dct_preds(line) for line in f]
                # preds[label.replace('txt', 'jpg')] = [self.dct_preds(i) for i in f.readlines()]
                
        return preds
        

In [7]:
yolo = YoloWrapper(weights='best.pt')

In [8]:
import os
import pathlib

test_path = pathlib.Path('./data/test/images')
samples = [str((test_path / file).resolve()) for file in os.listdir(test_path)[:5]]
preds = yolo.predict(samples)

YOLOv5 🚀 v7.0-90-ga895e98 Python-3.8.5 torch-1.13.1 CPU

Fusing layers... 


best.pt


Model summary: 157 layers, 7012822 parameters, 0 gradients
image 1/5 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1450_jpg.rf.e7ae062245bb8676d04fb109b2dd13b3.jpg: 256x256 1 vin, 131.8ms
image 2/5 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1494_jpg.rf.265a51023951f36169386ec7383d8b6b.jpg: 256x256 2 vins, 73.1ms
image 3/5 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1518_jpg.rf.07a3426d9ec6da2840d4af1bef33fb30.jpg: 256x256 1 vin, 73.6ms
image 4/5 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1626_jpg.rf.088d36226e07b8127851a1bb1b79e0a6.jpg: 256x256 1 vin, 77.6ms
image 5/5 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1720_jpg.rf.3471aa233437e81963497794f33fd312.jpg: 256x256 2 vins, 74.1ms
Speed: 0.3ms pre-process, 86.0ms inference, 0.8ms NMS per image

In [9]:
preds

{'image_1626_jpg.rf.088d36226e07b8127851a1bb1b79e0a6.jpg': [{'class_id': '0',
   'center_x': 0.607422,
   'center_y': 0.498047,
   'width': 0.316406,
   'height': 0.0429688,
   'conf': 0.74799}],
 'image_1450_jpg.rf.e7ae062245bb8676d04fb109b2dd13b3.jpg': [{'class_id': '0',
   'center_x': 0.539062,
   'center_y': 0.318359,
   'width': 0.453125,
   'height': 0.152344,
   'conf': 0.831326}],
 'image_1494_jpg.rf.265a51023951f36169386ec7383d8b6b.jpg': [{'class_id': '0',
   'center_x': 0.398438,
   'center_y': 0.501953,
   'width': 0.328125,
   'height': 0.0429688,
   'conf': 0.346608},
  {'class_id': '0',
   'center_x': 0.392578,
   'center_y': 0.521484,
   'width': 0.292969,
   'height': 0.0429688,
   'conf': 0.383392}],
 'image_1720_jpg.rf.3471aa233437e81963497794f33fd312.jpg': [{'class_id': '0',
   'center_x': 0.544922,
   'center_y': 0.566406,
   'width': 0.363281,
   'height': 0.0703125,
   'conf': 0.557337},
  {'class_id': '0',
   'center_x': 0.3125,
   'center_y': 0.0859375,
   'widt

## Dataset preparing

In [10]:
def get_full_path(path):
    return str(pathlib.Path(path).resolve())

get_full_path('./data/')

'/Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data'

In [11]:
def get_labels_for_img(img_path):
    prefix, _, file = img_path.rsplit('/', maxsplit=2)
    file_txt = removesuffix(file, ['.jpg', '.jpeg']) + '.txt'
    label_path = pathlib.Path(prefix) / 'labels' / file_txt

    with open(label_path, 'r') as f:
        labels = [YoloWrapper.dct_preds(line) for line in f]
    return labels

get_labels_for_img('./data/train/labels/rn707hrGaji74SaG1mSRqiZlsRYvZyqWyhFH9KF1_jpeg.rf.ffaae00a26a9dd2608dd54fde312b657.jpg')


[{'class_id': '0',
  'center_x': 0.4609375,
  'center_y': 0.498046875,
  'width': 0.787109375,
  'height': 0.087890625}]

In [12]:
import os

def prepare_dataset(data_path):
    res = {}
    data_path = pathlib.Path(data_path)
    res['X'] = [get_full_path(data_path/'images'/img_path) for img_path in os.listdir(data_path / 'images')]
    res['y_true'] = [get_labels_for_img(img_path) for img_path in res['X']]
    preds = yolo.predict(res['X'])

    res['y_pred'] = []
    for img_path in res['X']:
        _, img_name = img_path.rsplit('/', maxsplit=1)
        res['y_pred'].append(preds.get(img_name, [])) # maybe data was shuffled
    return res


In [15]:
# train = prepare_dataset('./data/train/')

YOLOv5 🚀 v7.0-90-ga895e98 Python-3.8.5 torch-1.13.1 CPU

Fusing layers... 


best.pt


Model summary: 157 layers, 7012822 parameters, 0 gradients
image 1/2797 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/train/images/00ZdDgWxsfKAlX2HuKrfmHknVJCXsOpEHfiZ3O1M_jpeg.rf.664834b733740c5cd7d724d8c16a73bb.jpg: 256x256 1 vin, 199.2ms
image 2/2797 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/train/images/01TtbfZAkO1v5oYqIr1uSnTHe2PgWuXjsIWaz1xL_jpeg.rf.a1f9dbf2f7f21bd2ce419bd1a5efc0a7.jpg: 256x256 1 vin, 93.1ms
image 3/2797 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/train/images/023Dq9mD5SKzvBwTONqrXB4noOepc5THJ5ZQ1hbG_jpeg.rf.af454bf9e812fa466c127f648624a659.jpg: 256x256 1 vin, 143.5ms
image 4/2797 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/train/images/05z46GIPw2SvMIq9Q4XsHiqOt7mKX19WsAvzbqnm_jpeg.rf.0cb8b39253a13c2862428258f7427539.jpg: 256x256 1 vin, 77.6ms
image 5/2797 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/train/image

In [16]:
# oos = prepare_dataset('./data/test/')

YOLOv5 🚀 v7.0-90-ga895e98 Python-3.8.5 torch-1.13.1 CPU



best.pt


Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
image 1/104 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1440_jpg.rf.f0d2f95fd5ceb2e2144be196b050b502.jpg: 256x256 1 vin, 82.6ms
image 2/104 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1441_jpg.rf.3cfb9e542144c004baafb57a034cfdd1.jpg: 256x256 1 vin, 91.4ms
image 3/104 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1442_jpg.rf.9428094378bd5805b83a4966337227db.jpg: 256x256 1 vin, 98.3ms
image 4/104 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1444_jpg.rf.7521d2202c73052e081130159b0549bb.jpg: 256x256 2 vins, 73.3ms
image 5/104 /Users/azatsultanov/Programming/vallib/repo/vallib/examples/yolo_test/data/test/images/image_1445_jpg.rf.c4081f1234efc17e83a09c3670573e18.jpg: 256x256 1 vin, 79.3ms
image 6/104 /Users/azatsultanov/Progr

In [17]:
# import pickle 

# with open('./train_dict.pkl', 'wb') as f:
#     pickle.dump(train, f)

# with open('./oos_dict.pkl', 'wb') as f:
#     pickle.dump(oos, f)

In [18]:
import pickle 

with open('./train_dict.pkl', 'rb') as f:
    train = pickle.load(f)

with open('./oos_dict.pkl', 'rb') as f:
    oos = pickle.load(f)

In [19]:
import sys

sys.path.append('../../')

In [21]:
from sbe_vallib.validation.sampler.supervised_sampler import SupervisedSampler

sampler = SupervisedSampler(train, oos)
sampler.set_state(seed=1, gen_method='bootstrap')
sampler.train['y_pred'][:2]

[[{'class_id': '0',
   'center_x': 0.447266,
   'center_y': 0.5,
   'width': 0.785156,
   'height': 0.09375,
   'conf': 0.778243}],
 [{'class_id': '0',
   'center_x': 0.605469,
   'center_y': 0.548828,
   'width': 0.460938,
   'height': 0.113281,
   'conf': 0.78405}]]

## Scorer

In [22]:
# cd external_libs/review_object_detection_metrics
# pip install .

In [46]:
from sbe_vallib.validation.scorer.base import BaseScorer
from review_object_detection_metrics.bounding_box import BoundingBox, BBFormat, BBType, CoordinatesType
from review_object_detection_metrics.evaluators.coco_evaluator import get_coco_summary

class ObjDetScorer(BaseScorer):
    def __init__(self):
        super().__init__()

    def score(self, X, y_true, y_pred, model, **kwargs):
        ground_truth = []
        detected = []
        for img_name, gt_boxes, det_boxes in zip(X, y_true, y_pred):
            for gt_box in gt_boxes:
                ground_truth.append(BoundingBox(
                    image_name=img_name,
                    class_id=gt_box['class_id'],
                    coordinates=[gt_box[i] for i in ['center_x', 'center_y', 'width', 'height']],
                    type_coordinates=CoordinatesType.RELATIVE,
                    img_size=model.img_size,
                    bb_type=BBType.GROUND_TRUTH,
                    format=BBFormat.XYWH
                ))
            for det_box in det_boxes:
                detected.append(BoundingBox(
                    image_name=img_name,
                    class_id=det_box['class_id'],
                    coordinates=[det_box[i] for i in ['center_x', 'center_y', 'width', 'height']],
                    type_coordinates=CoordinatesType.RELATIVE,
                    img_size=model.img_size,
                    confidence=det_box['conf'],
                    bb_type=BBType.DETECTED,
                    format=BBFormat.XYWH,
                ))
        return get_coco_summary(ground_truth, detected)
        


In [47]:
ObjDetScorer().score(sampler.train['X'], sampler.train['y_true'], sampler.train['y_pred'], yolo)

{'AP': 0.5089298121579297,
 'AP50': 0.948672332920254,
 'AP75': 0.4783974119706103,
 'APsmall': 0.4769984293703652,
 'APmedium': 0.5228687362370074,
 'APlarge': 0.3833200020902735,
 'AR1': 0.5885509838998211,
 'AR10': 0.5930948121645796,
 'AR100': 0.5930948121645796,
 'ARsmall': 0.5671739130434783,
 'ARmedium': 0.6001307759372276,
 'ARlarge': 0.5133333333333333}